In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import time

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS databricks_catalog.silver

### **Data Reading**

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@databricksetecharif.dfs.core.windows.net/customers")


In [0]:
df.display()

In [0]:
df = df.drop("_rescued_data")

In [0]:
df.display()

In [0]:
df1 = df.withColumn("domain",split(col("email"),"@")[1])

In [0]:
df1.display()

In [0]:
dd = df1.groupBy("domain").agg(count('customer_id').alias("total")).orderBy(desc("total")).limit(3)

In [0]:
dd.display()

In [0]:
df1.filter(col("email").contains("yahoo")).display()



In [0]:
df_gmail = df1.filter(col("email").contains("gmail"))
df_gmail.display()
time.sleep(5)

df_yahoo = df1.filter(col("email").contains("yahoo"))
df_yahoo.display()
time.sleep(5)

df_hotmail = df1.filter(col("email").contains("hotmail"))
df_hotmail.display()
time.sleep(5)


In [0]:
df_gmail.count()

In [0]:
if df_gmail.count() > df_yahoo.count():
  print("gmail is more popular")
else:
    print("yahoo is more popular")

In [0]:
df_yahoo.count()

In [0]:
df1.display()

In [0]:
df3 = df1.withColumn("full_name", concat((col("first_name")), lit(" "), (col("last_name"))))
df = df3.drop("first_name", "last_name")
display(df)



In [0]:
df.display()

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@databricksetecharif.dfs.core.windows.net/customers")

In [0]:
df_gmail = df1.filter(col("email").contains("gmail"))
df_gmail.display()

In [0]:
df_gmail3 = df1.where("email LIKE '%gmail%'")
df_gmail3.display()


In [0]:
df_gmail3 = df_gmail.select("customer_id","email","state") \
                     .where("email LIKE '%gmail%'")
df10 = df_gmail3.groupBy("state").agg(count("customer_id").alias("total")).orderBy(desc("total")).where((col("total") > 10) & (col("state") != "IA"))
df10.display()

### **Function**

In [0]:
df1.display()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_catalog.silver.customers_silver
USING DELTA
LOCATION 'abfss://silver@databricksetecharif.dfs.core.windows.net/customers'

In [0]:
%sql 
SELECT * FROM databricks_catalog.silver.customers_silver